<a href="https://colab.research.google.com/github/Joshpham244/VIB-credit-simul/blob/main/VIB_CREDIT_simul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/newdata/credit.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['amount'] = df['amount']*13346

In [ ]:
df['default'].value_counts()

In [ ]:
df = df.rename(columns={'default':'TARGET'})

In [ ]:
df['TARGET'].replace({2:1,1:0}, inplace=True)

In [ ]:
df.head()
#Deutsche Mark currency


In [ ]:
df['job'].value_counts()

In [ ]:
df['property'].value_counts()

In [ ]:
df['personal_status'].value_counts()

In [ ]:
df['credit_history'].value_counts()

In [ ]:
df['purpose'].value_counts()

In [ ]:
df['housing'].value_counts()

In [ ]:
df['existing_credits'].value_counts()

In [ ]:
df['employment_length'].value_counts()

In [ ]:
df['property'].value_counts()

In [ ]:
def chi2_by_hand(df, col1, col2):    
    #---create the contingency table---
    df_cont = pd.crosstab(index = df[col1], columns = df[col2])
    display(df_cont)
    #---calculate degree of freedom---
    degree_f = (df_cont.shape[0]-1) * (df_cont.shape[1]-1)
    #---sum up the totals for row and columns---
    df_cont.loc[:,'Total']= df_cont.sum(axis=1)
    df_cont.loc['Total']= df_cont.sum()
    print('---Observed (O)---')
    display(df_cont)
    #---create the expected value dataframe---
    df_exp = df_cont.copy()    
    df_exp.iloc[:,:] = np.multiply.outer(df_cont.sum(1).values,df_cont.sum().values) / df_cont.sum().sum()            
    print('---Expected (E)---')
    display(df_exp)
        
    # calculate chi-square values
    df_chi2 = ((df_cont - df_exp)**2) / df_exp    
    df_chi2.loc[:,'Total']= df_chi2.sum(axis=1)
    df_chi2.loc['Total']= df_chi2.sum()
    
    print('---Chi-Square---')
    display(df_chi2)
    #---get chi-square score---   
    chi_square_score = df_chi2.iloc[:-1,:-1].sum().sum()
     #---calculate the p-value---
    from scipy import stats
    p = stats.distributions.chi2.sf(chi_square_score, degree_f)
    return chi_square_score, degree_f, p

In [ ]:
chi2_by_hand(df, 'property', 'personal_status')

In [ ]:
df['employment_length'].replace({'1 - 4 yrs':'1-3_years', '> 7 yrs':'>5_years','4 - 7 yrs':'3-5_years',
                                 '0 - 1 yrs':'0-1_years'},inplace=True)

In [ ]:
df['employment_length'].value_counts()

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('Personal age and employment_length')
sns.boxplot(x=df.employment_length, y=df.age, palette=("cubehelix"))

plt.show()

In [ ]:
df = df.astype({"existing_credits":'category'})

df['existing_credits'].replace({1:'1_vib_card',2:'2_vib_card', 3:'another_bank', 4:'another_bank'}, inplace=True)

In [ ]:
df['dependents'].value_counts()

In [ ]:
df['installment_rate']

In [ ]:
df['credit_history'].value_counts()

In [ ]:
df['monthly_payment'] = (df['amount']*(df['installment_rate']/1200)*(1+df['installment_rate']/1200)**(df['months_loan_duration']))/((1+df['installment_rate']/1200)**(df['months_loan_duration'])-1)

In [ ]:
df['monthly_payment'] =round(df['monthly_payment'],0)

In [ ]:
df['monthly_payment']

In [ ]:
df['monthly_payment'].describe()

In [ ]:
df['monthly_payment'].hist()

In [ ]:
df['housing_time'] = round(df['age']/df['residence_history'],0)


In [ ]:
#moc hoc dai hoc 18-20 , moc tot nghiep 22-24, mốc get married (Đức)trung bình tuổi (31,5 nữ / 34,5 nam) ,  tầm tuổi trung niên(2-3 năm sau khi)
#xét thêm theo số lần di chuyển
#50 tuổi có ng phụ thuộc cao hơn 7 năm (nữ > 31, nam >41)
#25 tuổi tốt nghiệp đại học (ví dụ 30 tuổi -25 tuổi)


In [ ]:
df['residence_history'].describe()

In [ ]:
df['age'].describe()

In [ ]:
df['age'].hist()

In [ ]:
sns.regplot(x=df["age"], y=df["residence_history"],order=3, line_kws={"color":"orange"})

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('age vs residence history')
sns.boxplot(x=df.residence_history, y=df.age, palette=("cubehelix"))

plt.show()

In [ ]:
if (df['residence_history'] == 1).any():df['housing_time']=df['age'] 


In [ ]:
condition = [
             (df['age']<20),#Less than 1 year
             (df['age']>=20)&(df['age']<24), #1-3 years
             (df['age']>=24)&(df['age']<35),#3-6 years
             (df['age']>=35)&(df['age']<50),#6-10 years
             (df['age']>=50)#over 10 years
]
choice =['less_1_years', '1-3_years','3-6_years','6-10_years','over_10_years']
df['housing_time'] = np.select(condition, choice)

In [ ]:
df['housing_time'].describe()

In [ ]:
df['housing_time'].describe()

In [ ]:
df['housing_time'].value_counts()

In [ ]:
df['housing_time'].hist()

In [ ]:
df['age'].hist()

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('age vs child')
sns.boxplot(x=df.dependents, y=df.age, palette=("cubehelix"))

plt.show()

In [ ]:
theta = df[['age','residence_history']]

In [ ]:
df['residence_history'].hist()

In [ ]:
df['housing_time'].hist()

In [ ]:
df.head()

In [ ]:
def plot_kde_hist_var(df,varList,calcStat = True, drawAll = False):
    numVar = len(varList)

    plt.figure(figsize=(10,numVar*4))
    ks_stat_list = []
    ks_pval_list = []
    try:
        for i,var in enumerate(varList):
            tgt_true = df.loc[df['TARGET']==1,var]
            tgt_false = df.loc[df['TARGET']==0,var]

            # calculate statistical significance between both populations
            if calcStat == True:
                (ks_stat,ks_pval)= sp.stats.ks_2samp(tgt_true,tgt_false)
                ks_stat_list.append(ks_stat)
                ks_pval_list.append(ks_pval)
                ks_hval_list = [True for hyp in ks_pval_list if hyp<0.05]

            #
            median_tgt_true = tgt_true.median()
            median_tgt_false = tgt_false.median()
            corrVal = df['TARGET'].corr(df[var])
            print('Median Value of {} when Target (True): {:.6f}'.format(var,median_tgt_true))
            print('Median Value of {} when Target (False): {:.6f}'.format(var,median_tgt_false))
            print('Pearson Correlation of {} with Target (True): {:.6f}'.format(var,corrVal))

            # drawing KDE distributions
            tgt_true.dropna(inplace=True) # require to dropna for sns.distplot function
            tgt_false.dropna(inplace=True)
            plt.subplot(numVar,1,i+1)
            sns.distplot(tgt_true,rug=drawAll,kde=drawAll,label='Target: True')
            sns.distplot(tgt_false,rug=drawAll,kde=drawAll,label='Target: False')
            plt.legend()
            #plt.title(var)
    except TypeError as error:
        print(error)
        print('Features are objects.  Need ints/floats')

    return ks_hval_list, ks_pval_list

In [ ]:
df.describe()

In [ ]:
var_list=['months_loan_duration','amount','installment_rate','residence_history','age','existing_credits']

In [ ]:
hVal_list,pVal_list= plot_kde_hist_var(df,var_list)

In [ ]:
df['job'].value_counts()

In [ ]:
df_ext_src_corr1 = df[var_list].corr()
fig, ax = plt.subplots(figsize=(10,10))         # Sample figsize in inches
sns.heatmap(df_ext_src_corr1,vmin=-1.0,vmax=1.0,annot=True,ax= ax, linewidths=.5)

In [ ]:
ax = sns.scatterplot(x="months_loan_duration", y="amount", data=df)
ax.set_title("Concrete months loan duration vs amount")
ax.set_xlabel("months_loan_duration");

In [ ]:
sns.regplot(x=df["months_loan_duration"], y=df["amount"],order=3, line_kws={"color":"orange"})

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('age vs job')
sns.boxplot(x=df.job, y=df.age, palette=("cubehelix"))

plt.show()

In [ ]:
chi2_by_hand(df, 'credit_history', 'job')

In [ ]:
chi2_by_hand(df, 'purpose', 'job')

In [ ]:
df = df.drop(columns='purpose')

In [ ]:
df.head()

In [ ]:
df.shape


In [ ]:
conditions = [ 
              (df['job']=='skilled employee') ,
              (df['job']=='unskilled resident') ,
              (df['job']=='mangement self-employed') ,(df['job']=='unemployed non-resident')
]
values = ['>8mill','5-8mil','3-5mil','2-3mil']
      
df['Income'] = np.select(conditions, values)
df.head()
      

In [ ]:
np.random.seed(445)
df['education'] = np.random.choice(  
     a=['over_university_aboard', 'over_university_country', 'university','highschool'],  
    size=1000,  
     p=[0.1, 0.2, 0.5, 0.2] )  

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
credit_vib = df[['age','education', 'dependents','personal_status','housing','housing_time','employment_length','job','existing_credits','Income','monthly_payment','TARGET']]
credit_vib.head()

In [ ]:
credit_vib['Income'].value_counts()


# Scoring by domain

In [ ]:
Scoring_board = pd.DataFrame()

In [ ]:
conditions = [ 
              (df['age']<21) & (df['age']>=18),
              (df['age']<30) & (df['age']>=21),
              (df['age']>=30) & (df['age']<50),
              (df['age']>=60) & (df['age']<60)
]
values = [5, 6, 9, 8]
Scoring_board['AGE_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['education']=='university'),
              (df['education']=='over_university_country'),
              (df['education']=='highschool'),
              (df['education']=='over_university_aboard')
]
values = [7, 8, 5, 9]
      
Scoring_board['EDU_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['dependents']==0),
              (df['dependents']==1),
              (df['dependents']==2),
              (df['dependents']==3),
              (df['dependents']>=4)
]
values = [5, 8, 7, 6, 4]
      
Scoring_board['CHILD_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['personal_status']=='single male'),
              (df['personal_status']=='female'),
              (df['personal_status']=='married male'),
              (df['personal_status']=='divorced male')
]
values = [7, 7, 9, 6]
      
Scoring_board['FAM_STATUS_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
df['housing'].value_counts()

In [ ]:
conditions = [ 
              (df['housing']=='own'),
              (df['housing']=='rent'),
              (df['housing']=='for free'),
]
values = [9, 4, 6]
      
Scoring_board['HOUSING_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
df['housing_time'].value_counts()

In [ ]:
conditions = [ 
              (df['housing_time']=='less_1_years'),
              (df['housing_time']=='1-3_years'),
              (df['housing_time']=='3-6_years'),
              (df['housing_time']=='6-10_years'),
              (df['housing_time']=='over_10_years')
]
values = [6, 7, 8, 9, 10]
      
Scoring_board['HOUSING_TIME_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['employment_length']=='1-3_years'),
              (df['employment_length']=='>5_years'),
              (df['employment_length']=='3-5_years'),
              (df['employment_length']=='0-1_years'),
              (df['employment_length']=='unemployed')
]
values = [7, 9, 8, 6, 0]
      
Scoring_board['EMP_length_point'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['job']=='mangement self-employed'),
              (df['job']=='skilled employeet'),
              (df['job']=='unskilled resident'),
              (df['job']=='unemployed non-resident')
]
values = [9, 8, 7, 0]
      
Scoring_board['job_point'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['existing_credits']=='1_vib_card'),
              (df['existing_credits']=='2_vib_card'),
              (df['existing_credits']=='another_bank')
]
values = [7, 8, 6]
      
Scoring_board['existing_credits_point'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['Income']=='>8mill'),
              (df['Income']=='5-8mil'),
              (df['Income']=='3-5mil'),
              (df['Income']=='2-3mil')
]
values = [9, 8, 7, 5]
      
Scoring_board['Income_point'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
conditions = [ 
              (df['monthly_payment']<500000),
              (df['monthly_payment']>=500000)&(df['monthly_payment']<1000000),
              (df['monthly_payment']>=1000000)&(df['monthly_payment']<3000000),
              (df['monthly_payment']>=3000000)&(df['monthly_payment']<5000000),
              (df['monthly_payment']>=5000000)
]
values = [5, 6, 8, 9, 10]
      
Scoring_board['monthly_payment_POINT'] = np.select(conditions, values)
Scoring_board.head()
del conditions, values

In [ ]:
Scoring_board.head()

In [ ]:
Scoring_board.columns

In [ ]:
Scoring_board['credit_point'] = 0.04*Scoring_board['AGE_POINT'] + 0.05*Scoring_board['EDU_POINT'] + 0.05*Scoring_board['CHILD_POINT']+0.05*Scoring_board['FAM_STATUS_POINT']+0.15*Scoring_board['HOUSING_POINT']+0.03*Scoring_board['HOUSING_TIME_POINT']+0.04*Scoring_board['EMP_length_point']+0.05*Scoring_board['job_point']+0.05*Scoring_board['existing_credits_point']+0.35*Scoring_board['Income_point']+0.05*Scoring_board['monthly_payment_POINT']
                                 

In [ ]:
Scoring_board['credit_point'].hist()

In [ ]:
Scoring_board['credit_point'].describe()

In [ ]:
df['credit_point'] = Scoring_board['credit_point']

In [ ]:
df['TARGET']

In [ ]:
var_list1 = ['credit_point']
#varList = df.columns[41:56].values
hVal_list,pVal_list= plot_kde_hist_var(df,var_list1)

In [ ]:
conditions = [ 
              (Scoring_board['credit_point']>7),
              (Scoring_board['credit_point']<=7)
              
]
values = [0, 1]
      
df['ESTIM_TARGET'] = np.select(conditions, values)
df.head()
del conditions, values

In [ ]:
df.ESTIM_TARGET.value_counts()

In [ ]:
df = df.groupby('TARGET', group_keys=False).apply(lambda x: x.sample(frac=0.2))

In [ ]:
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
print(classification_report(df.TARGET, df.ESTIM_TARGET))

In [ ]:
domain_fpr, domain_tpr, domain_threshold = metrics.roc_curve(df.TARGET, df.ESTIM_TARGET)
domain_roc_auc = metrics.auc(domain_fpr, domain_tpr)

import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(domain_fpr, domain_tpr, '^', label='domain (AUC = %0.2F)' % domain_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Machine learing scoring

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('Income vs payment')
sns.boxplot(x=df.Income, y=df.monthly_payment, palette=("cubehelix"))

plt.show()

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('Personal housing vs payment')
sns.boxplot(x=df.TARGET, y=df.monthly_payment, palette=("cubehelix"))

plt.show()

In [ ]:
credit_vib.drop(columns='job',inplace=True)
credit_vib.head()

In [ ]:
chi2_by_hand(credit_vib, 'personal_status', 'housing')

In [ ]:
chi2_by_hand(credit_vib, 'personal_status', 'dependents')

In [ ]:
credit_vib.drop(columns='personal_status',inplace=True)

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('Personal housing time and Target')
sns.boxplot(x=df.TARGET, y=df.housing_time, palette=("cubehelix"))

plt.show()

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,2)
plt.title('Personal age and Target')
sns.boxplot(x=df.TARGET, y=df.age, palette=("cubehelix"))

plt.show()

In [ ]:
credit_vib = pd.get_dummies(credit_vib, drop_first= False)

In [ ]:
print('simul dataset shape: '.format(0),df.shape)


In [ ]:
credit_vib.head()

In [ ]:
df_ext_src_corr1 = credit_vib[['age','dependents','monthly_payment']].corr()
fig, ax = plt.subplots(figsize=(10,5))         # Sample figsize in inches
sns.heatmap(df_ext_src_corr1,vmin=-1.0,vmax=1.0,annot=True,ax= ax, linewidths=.5)

In [ ]:
label = credit_vib['TARGET']
df_train = credit_vib.copy()
df_train = df_train.drop(columns=['TARGET'])

In [ ]:
Y = label.copy()
X = df_train.copy()
feat_names = list(X.columns)

X_ids = X.index.values
Y_ids = Y.index.values

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
feature_importances = np.zeros(X.shape[1])

model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 1000, class_weight = 'balanced')

In [ ]:
for i in range(2):
    
    # Split into training and validation set
    train_features, valid_features, train_y, valid_y = train_test_split(X, Y, test_size = 0.25, random_state = i)
    
    # Train using early stopping
    model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
              eval_metric = 'auc', verbose = 200)
    
    # Record the feature importances
    feature_importances += model.feature_importances_

In [ ]:
feature_importances = feature_importances / 2

feature_importances = pd.DataFrame({'feature': feat_names, 'importance': feature_importances}).sort_values('importance', ascending = False)

feature_importances.head()

In [ ]:
zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
print('There are %d features with 0.0 importance' % len(zero_features))
feature_importances.tail()

In [ ]:
def plot_feature_importances(df, threshold = 0.9):
    """
    Plots 15 most important features and the cumulative importance of features.
    Prints the number of features needed to reach threshold cumulative importance.
    
    Parameters
    --------
    df : dataframe
        Dataframe of feature importances. Columns must be feature and importance
    threshold : float, default = 0.9
        Threshold for prining information about cumulative importances
        
    Return
    --------
    df : dataframe
        Dataframe ordered by feature importances with a normalized column (sums to 1)
        and a cumulative importance column
    
    """
    
    plt.rcParams['font.size'] = 18
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    # Cumulative importance plot
    plt.figure(figsize = (8, 6))
    plt.plot(list(range(len(df))), df['cumulative_importance'], 'r-')
    plt.xlabel('Number of Features'); plt.ylabel('Cumulative Importance'); 
    plt.title('Cumulative Feature Importance');
    plt.show();
    
    importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
    print('%d features required for %0.2f of cumulative importance' % (importance_index + 1, threshold))
    
    return df

In [ ]:
norm_feature_importances = plot_feature_importances(feature_importances)


In [ ]:
df_train = df_train.drop(columns= zero_features)

In [ ]:
def identify_zero_importance_features(train, train_labels, iterations = 2):
# Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])

    # Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')
    
    # Fit the model multiple times to avoid overfitting
    for i in range(iterations):

        # Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)

        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
                  eval_metric = 'auc', verbose = 200)

        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    
    feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
    
    # Find the features with zero importance
    zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    
    return zero_features, feature_importances

In [ ]:
threshold = 0.95

# Extract the features to keep
features_to_keep = list(norm_feature_importances[norm_feature_importances['cumulative_importance'] < threshold]['feature'])

# Create new datasets with smaller features
df_new = df_train[features_to_keep]

In [ ]:
df_new['TARGET'] = Y


In [ ]:
df_new.head()

In [ ]:
df_new['TARGET'].value_counts()

In [ ]:
df_ext_src = df_new[['monthly_payment','age','TARGET']] 


In [ ]:

df_ext_src_sample = df_ext_src
grid = sns.PairGrid(data = df_ext_src_sample, diag_sharey=True,
                    hue = 'TARGET', 
                    vars = [x for x in list(df_ext_src_sample.columns) if x != 'TARGET'])

grid.map_upper(plt.scatter, alpha = 0.2)
grid.map_diag(sns.kdeplot)
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

In [ ]:
Xnew_train =df_new.copy()

In [ ]:
Xnew_train = df_new.drop(columns=['TARGET','age'])

In [ ]:
df_new['age']

In [ ]:
Xnew_train.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(Xnew_train, Y, test_size=.2,
                                                    random_state=0)

X_train_ids = X_train.index.values
X_test_ids = X_test.index.values

feat_names = list(Xnew_train.columns)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0,1))

In [ ]:
scaler.fit(Xnew_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.6)
under = RandomUnderSampler(sampling_strategy=0.8)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb

In [ ]:
clf = XGBClassifier(objective='binary:logistic',seed=42,scale_pos_weight=2.2775)

In [ ]:
clf.fit(X_train, Y_train, verbose=True, early_stopping_rounds=10, eval_metric='aucpr', eval_set=[[X_test,Y_test]])

In [ ]:
pred_proba_t4 = clf.predict_proba(X_test)[:,1]


In [ ]:
score = clf.score(X_test, Y_test)

pred_proba_t33 = clf.predict_proba(X_test)

print("Testing Acc:", score)

y_pred4 = clf.predict(X_test)
cm = confusion_matrix(Y_test, y_pred4)

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm)
ax.grid(False)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
plt.show()

In [ ]:
!pip install optuna

In [ ]:
def optimize(trial):
  objective = trial.suggest_categorical("objective",["multi:softmax","multi:softprob"])
  eval_metric = trial.suggest_categorical("eval_metric",["merror","mlogloss","auc"])
  max_depth = trial.suggest_int("max_depth", 1, 50)
  eta = trial.suggest_float("learning_rate",  0.1, 1, log=True)
  gamma = trial.suggest_float("gamma", 0.1, 1 ,log=True)
  reg_lambda = trial.suggest_float("reg_lambda", 0.1, 1 ,log=True)
  #scale_pos_weight = trial.suggest_float("scale_pos_weight", 1, 100, log=True)
  model = XGBClassifier(max_depth=max_depth, eta=eta , gamma=gamma, reg_lambda=reg_lambda, objective = objective, booster= "gbtree",
                        sampling_method ='gradient_based', num_class=2)#scale_pos_weight=scale_pos_weight
  accuracy_area = []
  model.fit(X_train,Y_train)
  preds = model.predict(X_test)
  fold_acc = metrics.accuracy_score(Y_test, preds)
  accuracy_area.append(fold_acc)
  return -1.0 * np.mean(accuracy_area)

In [ ]:
import optuna
from sklearn import  model_selection

In [ ]:
#study = optuna.create_study(direction="minimize")
#study.optimize(optimize, n_trials=1000)

In [ ]:
parameters: {'objective': 'multi:softmax', 'eval_metric': 'mlogloss', 'max_depth': 39, 'learning_rate': 0.1598668143191308, 'gamma': 0.9613463846501532, 'reg_lambda': 0.31206454610995665}

In [ ]:
parameters: {'objective': 'multi:softprob', 'eval_metric': 'mlogloss', 'max_depth': 1, 'learning_rate': 0.49221976744492657, 'gamma': 0.8807972671374964, 'reg_lambda': 0.17934179652902277}

In [ ]:
clf2 = XGBClassifier(objective='multi:softprob', sampling_method ='gradient_based',booster='gbtree', seed=42, max_depth=1, learning_rate= 0.49221976744492657, gamma= 0.8807972671374964, reg_lambda=0.17934179652902277, num_class=2)

In [ ]:
clf2.fit(X_train, Y_train, verbose=True, early_stopping_rounds=10, eval_metric='mlogloss', eval_set=[[X_test, Y_test]])

In [ ]:
score2 = clf2.score(X_test, Y_test)
pred_proba_t2 = clf2.predict_proba(X_test)


y_pred2 = clf2.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Cross-validate on the train data
train_cv = cross_val_score(X=X_train,y=Y_train,estimator=clf2,cv=5)
print("TRAIN GROUP")
print("\nCross-validation accuracy scores:",train_cv)
print("Mean score:",train_cv.mean())

train_precision_cv = cross_val_score(X=X_train,y=Y_train,estimator=clf2,cv=5, scoring='precision')
print("TRAIN GROUP")
print("\nCross-validation precision scores:",train_precision_cv)
print("Mean score:",train_precision_cv.mean())

train_recall_cv = cross_val_score(X=X_train,y=Y_train,estimator=clf2,cv=5, scoring='recall')
print("TRAIN GROUP")
print("\nCross-validation recall scores:",train_recall_cv)
print("Mean score:",train_recall_cv.mean())


train_roc_auc_cv = cross_val_score(X=X_train,y=Y_train,estimator=clf2,cv=5, scoring='roc_auc')
print("TRAIN GROUP")
print("\nCross-validation roc_auc scores:",train_roc_auc_cv)
print("Mean score:",train_roc_auc_cv.mean())

In [ ]:
print(classification_report(Y_test, y_pred2))


In [ ]:
pred_proba_t5 = clf2.predict_proba(X_test)[:,1]


In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification


clf3_fpr, clf3_tpr, clf3_threshold = metrics.roc_curve(Y_test, pred_proba_t5)

#rf_fpr, rf_tpr, rf_threshold = metrics.roc_curve(Y_test, y_test_pred)

#rf_roc_auc = metrics.auc(rf_fpr, rf_tpr)
clf3_roc_auc = metrics.auc(clf3_fpr, clf3_tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')


plt.plot(clf3_fpr, clf3_tpr, '*', label='XGB- (AUC = %0.2F)' % clf3_roc_auc)

#plt.plot(rf_fpr, rf_tpr, '--', label='rf (AUC = %0.2F)' % rf_roc_auc)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
features = Xnew_train.columns
importances = clf2.feature_importances_
indices = np.argsort(importances)

# customized number 
num_features = 10 

plt.figure(figsize=(10,8))
plt.title('Feature Importances')

# only plot the customized number of features
plt.barh(range(num_features), importances[indices[-num_features:]], color='b', align='center')
plt.yticks(range(num_features), [features[i] for i in indices[-num_features:]])
plt.xlabel('Relative Importance')
plt.show()